In [1]:
import importlib
import pandas as pd
import nltk
import gensim 
import logging
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

import utils
importlib.reload(utils)

import keras
from keras import preprocessing
from keras.models import Sequential
from keras.layers import Flatten, Dense, Embedding

Using TensorFlow backend.


In [2]:
data = pd.read_csv("data/labeledTrainData.tsv", delimiter="\t", quoting=3)
train_data, test_data = train_test_split(data, test_size=0.25)

In [3]:
# Clean the text
train_data['review_cleaned'] = train_data.review.apply(lambda x: utils.normalize_text(x))
test_data['review_cleaned'] = test_data.review.apply(lambda x: utils.normalize_text(x))

unlabel_data = pd.read_csv("data/testData.tsv", header=0, delimiter="\t", quoting=3 )
unlabel_data['review_cleaned'] = unlabel_data.review.apply(lambda x: utils.normalize_text(x))

/home/andryw/Projects/tutorials/python_env/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/andryw/Projects/tutorials/python_env/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [4]:
def get_total_words(text_list):
    text_set = set()
    for text in text_list:
        text_set |= set(text.split())
    return len(text_set)

def get_max_size_of_sentence(text_list):
    max_size = 0
    for text in text_list:
        max_size = max(max_size, len(text))
    return max_size

In [5]:
## Find the total of unique words
total_words = get_total_words(list(train_data['review_cleaned']) + 
                list(test_data['review_cleaned']) +
                list(unlabel_data['review_cleaned']))

## Find max size of a setence
max_size = get_max_size_of_sentence(list(train_data['review_cleaned']) + 
                list(test_data['review_cleaned']) +
                list(unlabel_data['review_cleaned']))

tokenizer = preprocessing.text.Tokenizer(total_words)

In [6]:
tokenizer.fit_on_texts(list(train_data['review_cleaned']) + 
                       list(test_data['review_cleaned']) +
                       list(unlabel_data['review_cleaned'])
                      )

In [7]:
train_one_hot_index = tokenizer.texts_to_sequences(list(train_data['review_cleaned']))
train_one_hot_index = preprocessing.sequence.pad_sequences(train_one_hot_index, max_size)

test_one_hot_index = tokenizer.texts_to_sequences(list(test_data['review_cleaned']))
test_one_hot_index = preprocessing.sequence.pad_sequences(test_one_hot_index, max_size)

unlabel_data_one_hot_index = tokenizer.texts_to_sequences(list(unlabel_data['review_cleaned']))
unlabel_data_one_hot_index = preprocessing.sequence.pad_sequences(unlabel_data_one_hot_index, max_size)

In [8]:
x_train = train_one_hot_index
y_train = train_data.sentiment

x_test = test_one_hot_index
y_test  = test_data.sentiment

In [16]:
model = Sequential()
model.add(Embedding(input_dim=total_words, output_dim=20, input_length=max_size)) # samples x maxlen => samples x maxlen x 8 
model.add(Flatten()) # samples x maxlen x 8 => samples x maxlen*8
# model.add(Dense(10, activation="relu"))
model.add(Dense(1, activation="sigmoid"))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 9434, 20)          2024920   
_________________________________________________________________
flatten_3 (Flatten)          (None, 188680)            0         
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 188681    
Total params: 2,213,601
Trainable params: 2,213,601
Non-trainable params: 0
_________________________________________________________________


In [17]:
model.compile(optimizer=keras.optimizers.Adam(lr=1e-3), loss='binary_crossentropy', metrics=['acc'])

In [18]:
model.fit(x_train, y_train, epochs=1, validation_data=(x_test, y_test))

Train on 18750 samples, validate on 6250 samples
Epoch 1/1
18750/18750 [==============================] - 22s 1ms/step - loss: 0.4531 - acc: 0.7789 - val_loss: 0.2916 - val_acc: 0.8829


## Predict

In [19]:
prediction = [x[0] for x in model.predict_classes(unlabel_data_one_hot_index)]
utils.save_predict(unlabel_data, 
                   prediction, 
                   "keras_embedding_2-layer_20-dim_1-epoch.csv")